In [15]:
import yaml
import sys
import spacy
import ujson

import pandas as pd
import numpy as np
import rubrix as rb

from yaml import Loader

sys.path.insert(0, '../')
from ids import rubrix_api_key
from tqdm.auto import tqdm
from collections import defaultdict

pd.set_option('display.max_colwidth', 100)
rb.init("http://localhost:6900/", api_key=rubrix_api_key)

# Load users
users = yaml.load(open('../.users.yaml','r'), Loader)
active_users = set([u['username'] for u in users if 'credits' in u])
u2cred = {u['username']: u['credits'] for u in users if 'credits' in u}

nlp = spacy.load('en_core_sci_lg')

In [18]:
correct_pmids = set(open('../pmid_logs/correct.txt', 'r').read().strip().split())
filtered_abstracts = []
seen = set([])
for x in ujson.load(open('../parsed.json','r')):
    if x['metadata']['pmid'] in correct_pmids:
        if x['metadata']['pmid'] not in seen:
            filtered_abstracts.append(x)
            seen.add(x['metadata']['pmid'])

for x in tqdm(filtered_abstracts):
    doc = nlp(x['text'])
    x['tokens'] = [token.text for token in doc]

records = [rb.TokenClassificationRecord(
            text=x['text'],
            tokens=x['tokens'],
            metadata=x['metadata'],
            # prediction=entities,
            # prediction_agent="en_core_web_trf",
            ) for x in filtered_abstracts
            ]
        


  0%|          | 0/1353 [00:00<?, ?it/s]

/nethome/dkartchner3/.local/lib/python3.9/site-packages/rubrix/client/models.py:39: UserWarning: Some metadata values exceed the max length. Those values will be truncated by keeping only the last 128 characters.
  warnings.warn(


In [19]:
# Logging into Rubrix
rb.log(
    records=records,
    name="pico_1",
    tags={
        "task": "PICO",
        "family": "token-classification",
        "dataset": "pico_round_1",
    },
)

  0%|          | 0/1353 [00:00<?, ?it/s]

1353 records logged to http://localhost:6900/ws/davidkartchner/pico_1


BulkResponse(dataset='pico_1', processed=1353, failed=0)

In [22]:
text = "We hypothesized that Vitamin D3 may play a role in Alzheimer's Disease. A cohort of 330 otherwise healthy AD patients took 5000 iu of Vitamin D3 per day for 10+ years preceding their diagnosis.  Their median Montreal cognitive assessment score (MOCA) at AD diagnosis was 17.7 +/- 2.6.  An age and sex-matched control cohort of 442 AD patients who had never taken a Vitamin D3 supplement had a median MOCA of 14.3 +/- 2.9.  Kruskal-wallis analysis illustrated that previous Vitamin D3 therapy was associated with a significant increase in baseline MOCA (p = 0.034).  Vitamin D3 may provide a protective effect in the etiology of Alzheimer's."
doc = nlp(text)
tokens = [token.text for token in doc]
records = [rb.TokenClassificationRecord(
            text=text,
            tokens=tokens,
            # metadata=x['metadata'],
            # prediction=entities,
            # prediction_agent="en_core_web_trf",
            )]

# Logging into Rubrix
rb.log(
    records=records,
    name="pico_1",
    tags={
        "task": "PICO",
        "family": "token-classification",
        "dataset": "pico_round_1",
    },
)

  0%|          | 0/1 [00:00<?, ?it/s]

ConnectError: [Errno 111] Connection refused